In [ ]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai

In [ ]:
project_id = "quectonic"
display_name = "sample 1"
paths = ["gs://hl-txt"]

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
vertexai.init(project=project_id, location="us-central1")

In [ ]:
rag_corpus = rag.create_corpus(display_name=display_name)

In [ ]:
response = rag.import_files(
    rag_corpus.name,
    paths,
    chunk_size=128,
    chunk_overlap=32,
)

In [ ]:
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name
        )
    ],
    text="Chlamydia",
    similarity_top_k=3,  # Optional
    vector_distance_threshold=0.5,  # Optional
)

In [ ]:
print(response)

contexts {
  contexts {
    source_uri: "gs://hl-txt/abdominal-pain-and-painful-urination_utm_source_ReadNext_c002c959-eb87-4c72-b967-335200e96509.txt"
    text: "People who have chlamydia often do not have outward symptoms in the early stages. Genital herpes Genital herpes is a sexually transmitted disease (STD). This STD causes herpetic sores, which are painful blisters (fluid-filled bumps) that can break open and ooze fluid."
    distance: 0.36795848483094096
  }
  contexts {
    source_uri: "gs://hl-txt/abdominal-pain-and-painful-urination_utm_source_ReadNext_c002c959-eb87-4c72-b967-335200e96509.txt"
    text: "Prostatitis Prostatitis is inflammation of your prostate gland. The inflammation may spread to the area around your prostate. Chlamydia infection Chlamydia is a common sexually transmitted infection (STI) caused by bacteria. People who have chlamydia often do not have outward symptoms in the early stages."
    distance: 0.38634342410892786
  }
  contexts {
    source_uri: "g

In [ ]:
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name
                )
            ],
            similarity_top_k=3,
            vector_distance_threshold=0.5,
        )
    )
)

In [ ]:
system_instruction = [
    "You are a clinical educator that provides a patient with health related information based on their health data and incoming concern queries.",
    "You will also suggest the severity of the disease based on the conclusion you provide to the patient and recommend the appropriate next steps and when to seek official medical attention.",
    "You want to avoid recommending the patient to visit a medical facility unless it is necessary.",
    """You want to analyze with the following workflow:
      ### Input
      [Patient's concerns with symptoms]

      ### Thought Process
      [Based on patient's symptoms, look for direct cause of the symptoms. Repeat this steps until a diagnosis is reached]

      ### Action
      [Provide simple explanation of the medical concerns patient inquired, the overall severity and next step approaches]
    """,
    """You want to answer with the following format:
      Based on the information provided, {simple explanation of patient's medical concerns}
      Severity: {overall severity}
      Next Steps: {next steps}
    """
    ]


model = GenerativeModel(
    model_name="gemini-1.0-pro-002", tools=[rag_retrieval_tool], system_instruction=system_instruction,
)

dumb_model = GenerativeModel(
    model_name="gemini-1.0-pro-002", tools=[], system_instruction=system_instruction,
)

critical_param_message = "Heart Rate Fluctuation based on chronic tracking does not show abnormal fluctuations. 8 hours sleep was accomplished daily. Last Blood Test (2 days ago) showed positive for chlamydia and negative for other STIs. No significant weight loss."

In [ ]:
# Retrieval || Additional Parameters
response = model.generate_content(f"I have been throwing up and my period stopped. My titties also look swollened! {critical_param_message} What happened to me? Give me a detailed and step by step answer.")
print(response.text)

Based on the information provided, it is possible that you are experiencing symptoms of pelvic inflammatory disease (PID). PID is a serious infection of the female reproductive organs that can occur when bacteria from the vagina or cervix spread to the uterus, fallopian tubes, and ovaries. PID can cause a variety of symptoms, including abdominal pain, fever, vaginal discharge, and irregular bleeding.

Severity: PID is a serious condition that can lead to long-term health problems if it is not treated promptly. The symptoms you are experiencing are consistent with PID, and it is important to see a doctor as soon as possible to get a diagnosis and treatment.

Next Steps: If you think you might have PID, see a doctor as soon as possible. Your doctor will be able to do a pelvic exam and take a swab to check for the bacteria that cause PID. They may also order other tests, such as an ultrasound or blood tests, to rule out other possible causes of your symptoms. Treatment for PID usually inv

In [ ]:
# No Retrieval || No Additional Parameters
response = dumb_model.generate_content("I have been throwing up and my period stopped. My titties also look swollened! What happened to me? Give me a detailed and step by step answer.")
print(response.text)

Based on the information provided, you might be pregnant. 

**Severity:** Moderate

**Next Steps:**

1. **Take a home pregnancy test.** This is the first step to confirm if you are pregnant. Home pregnancy tests are readily available at most drugstores and grocery stores.
2. **If the home pregnancy test is positive, schedule an appointment with your doctor or healthcare provider.** They will be able to confirm the pregnancy and discuss your options.
3. **If the home pregnancy test is negative, you may want to consider other possible causes of your symptoms, such as a stomach virus or food poisoning.** You can also talk to your doctor or healthcare provider about your symptoms.

**Additional Information:**

* **Morning sickness** is a common symptom of early pregnancy. It can cause nausea, vomiting, and food aversions.
* **Breast tenderness** is another common symptom of early pregnancy. Your breasts may feel swollen, sore, or tingly.
* **A missed period** is the most common sign of pre

In [ ]:
# No Retrieval || Additional Parameters
response = dumb_model.generate_content(f"I have been throwing up and my period stopped. My titties also look swollened! {critical_param_message} What happened to me? Give me a detailed and step by step answer.")
print(response.text)

## Based on the information provided, you may be experiencing the following:

**1. Pregnancy:** Nausea and vomiting are common symptoms of early pregnancy. Additionally, breast swelling and tenderness are also common early pregnancy symptoms. A missed period can be another sign of pregnancy. 

**2. Ectopic Pregnancy:** This is a serious condition that occurs when a fertilized egg implants outside of the uterus, usually in the fallopian tube. Ectopic pregnancies can cause severe pain, vaginal bleeding, and other symptoms.

**3. Pelvic Inflammatory Disease (PID):** This is an infection of the female reproductive organs that can be caused by chlamydia. PID can cause pain in the lower abdomen, fever, and abnormal vaginal discharge.

**4. Other medical conditions:** There are a number of other medical conditions that can cause nausea, vomiting, breast swelling, and missed periods. These include:

* **Gastroenteritis:** This is an inflammation of the stomach and intestines that can be caused

In [ ]:
# Retrieval || No Additional Parameters
response = model.generate_content("I have been throwing up and my period stopped. My titties also look swollened! What happened to me? Give me a detailed and step by step answer.")
print(response.text)

Based on your symptoms, it is possible that you are pregnant. Pregnancy can cause a variety of symptoms, including nausea, vomiting, breast tenderness, and changes in your menstrual cycle. If you are concerned that you may be pregnant, it is important to see a doctor to get a definitive diagnosis.

However, it is also important to note that there are other possible explanations for your symptoms. For example, you could have a stomach virus, food poisoning, or another medical condition. If your symptoms are severe or you are concerned about them, it is always best to see a doctor to get a diagnosis and treatment plan.

Here are some next steps you can take:

1. Take a home pregnancy test. This is the quickest and easiest way to find out if you are pregnant.
2. See a doctor. If you are pregnant or think you might be pregnant, it is important to see a doctor to get prenatal care.
3. Rule out other possible causes of your symptoms. If you are not pregnant, there are other possible explanat